In [1]:
pip install transformers==3.0.2 sentence_transformers==0.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 769 kB 26.5 MB/s 
     |████████████████████████████████| 65 kB 1.8 MB/s 
     |████████████████████████████████| 3.0 MB 31.2 MB/s 
     |████████████████████████████████| 880 kB 38.3 MB/s 
     |████████████████████████████████| 1.3 MB 34.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-py3-none-any.whl size=97300 sha256=b4ecad11a22a187be2e94a96c3178fe6715a09f1f42e690de99a92bd4561e1e9
  Stored in directory: /root/.cache/pip/wheels/9e/03/96/a32af02b51743e0f3812c48cfc6d37ee8a6eaf6706b6014876
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3b9c966b0154076bd53f1f3975ded0cbd381529686d8172772a7dbfd5f90e5a7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sacremoses


In [5]:
import re
import os
import json
import pickle
import numpy as np
import pandas as pd
import random
import torch 
from torch import nn
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
cd /content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/phrase-bert-topic-model-master/phrase-topic-model/

/content/drive/.shortcut-targets-by-id/1zZCiCdXv6HLRAkEj-TagubvK1e49OsAz/Capstone - Causal Narratives Extraction/Phrase BERT code/phrase-bert-topic-model-master/phrase-topic-model


In [11]:
from model.dae_model import DictionaryAutoencoder
from model_utils import run_epoch, text_to_topic, rank_topics_by_percentage

In [3]:
cd /content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/

/content/drive/.shortcut-targets-by-id/1zZCiCdXv6HLRAkEj-TagubvK1e49OsAz/Capstone - Causal Narratives Extraction/Phrase BERT code


not changing the variable name "headline" from now on to save time, but the file is actually ones_NYT_2019.csv

In [8]:
data = pd.read_csv('ones_NYT_2019.csv')[['text', 'args']]
data.head(10)

,text,args
0,Serious economists warned that this growth was...,('driven by the shift from fiscal austerity to...
1,"In particular, the uncertainty may be deterrin...","('In', 'business spending.')"
2,"In particular, the uncertainty may be deterrin...","('the uncertainty may be deterring', 'business..."
3,George Wallace spent most of life trying to ho...,('trying to hold back the tide of change that ...
4,"To defenders of the old order, America had don...","('To defenders of the old order, America had d..."
5,"But to many others, seeking a broader form of ...","('others,', 'Wilson’s rhetoric seemed to descr..."
6,"But to many others, seeking a broader form of ...","('seeking a broader form of democracy,', 'But ..."
7,"But to many others, seeking a broader form of ...","('seeking a broader form of democracy,', 'Wils..."
8,A vigorous government response led to the crea...,"('A vigorous government response', 'the creati..."
9,"A retaliatory bomb, directed at Palmer, explod...","('A retaliatory bomb, directed at Palmer, expl..."


In [31]:
data.iloc[90,1]

"('Parliament ultimately rejects Mrs. May’s proposal,', 'investors will probably remain nervous.')"

In [32]:
data.iloc[90,0]

'If Parliament ultimately rejects Mrs. May’s proposal, investors will probably remain nervous.'

In [13]:
headline = pd.read_csv('ones_NYT_2019.csv')
data_args = headline["args"].str.split("', '", n = 1, expand = True)
headline["arg0"]= data_args[0].apply(lambda x : '' if x is None else x.strip("('").strip())
headline["arg1"]= data_args[1].apply(lambda x : '' if x is None else x.strip("')"))
headline.loc[headline["arg1"] != headline["arg1"],"arg1"] = 'NA'
headline.loc[headline["arg0"] != headline["arg0"],"arg0"] = 'NA'

In [14]:
headline.head(5)

,id,index,outlet,political_leaning,date_publish,text,args,year,arg0,arg1
0,4075871,polusa_4075871_1_1,The New York Times,LEFT,2019-01-01 00:00:00,Serious economists warned that this growth was...,('driven by the shift from fiscal austerity to...,2019,driven by the shift from fiscal austerity to w...,that this growth was a temporary lift — a “sug...
1,4075871,polusa_4075871_6_0,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('In', 'business spending.')",2019,In,business spending.
2,4075871,polusa_4075871_6_1,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('the uncertainty may be deterring', 'business...",2019,the uncertainty may be deterring,business spending.
3,4083565,polusa_4083565_5_0,The New York Times,LEFT,2019-01-01 00:00:03,George Wallace spent most of life trying to ho...,('trying to hold back the tide of change that ...,2019,trying to hold back the tide of change that Ro...,George Wallace spent most of life
4,4083565,polusa_4083565_9_0,The New York Times,LEFT,2019-01-01 00:00:03,"To defenders of the old order, America had don...","('To defenders of the old order, America had d...",2019,"To defenders of the old order, America had don...",and there was no need to open the floodgates.


In [15]:
# list of arg0s and arg1s
headline_arg0s = headline['arg0']
headline_arg1s = headline['arg1']
headline_arg0s = headline_arg0s.tolist()
headline_arg1s = headline_arg1s.tolist()
# list of all args for POLUSA headlines
headline_args = headline_arg0s + headline_arg1s

In [16]:
# construct text_list
headline_text = headline['text']
headline_text_list = headline_text.tolist()

In [17]:
# construct dictionaries of word2id
headline_word2id = {val : idx for idx, val in enumerate(set(headline_args))}
len(headline_word2id.keys()), len(headline_args) # duplicates

(76078, 112264)

In [18]:
# construct dictionaries of id2word
headline_id2word = {val: key for key, val in headline_word2id.items()}

In [19]:
# contruct dictionaries of id2freq
headline_id2freq = headline_id2word.copy()
headline_freq = [(arg, headline_args.count(arg)) for arg in set(headline_args)]
i = 0
for key, val in headline_id2freq.items():
    headline_id2freq[key] = headline_freq[i][1]
    i += 1

In [21]:
# load the Phrase-BERT model through the sentence-BERT interface
model_path = "/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/pooled_context_para_triples_p=0.8/"
model = SentenceTransformer(model_path)

In [22]:
# compute phrase embeddings using Phrase-BERT
headline_phrase_embs = model.encode(set(headline_args), batch_size=8, show_progress_bar=True)
headline_embs = np.asarray(headline_phrase_embs)

Batches:   0%|          | 0/9510 [00:00<?, ?it/s]

In [23]:
# save the results
topic_model_data_path = "/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/"
np.save(os.path.join(topic_model_data_path, 'ones_NYT_2019_embs_matrix_np'), headline_embs)

In [24]:
# set seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [32]:
topic_model_data_path = "/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/"

In [33]:
headline_embs_matrix_np = np.load(os.path.join(topic_model_data_path, f"ones_NYT_2019_embs_matrix_np.npy"))
print(f"Loaded headline word embedding from {topic_model_data_path}")
print(f"Loaded vocab size of {len(headline_word2id)} (including phrases)")

Loaded headline word embedding from /content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/
Loaded vocab size of 76078 (including phrases)


In [34]:
len(headline_embs_matrix_np)

76078

Below code is adapted from Phrase-Bert: https://github.com/sf-wa-326/phrase-bert-topic-model

In [35]:
# word frequency and filter info

# compute the length (in n-grams)
# setting word_threshould really high to include every phrase, reset to lower value to remove longer phrases
word_threshold = 100

headline_len_words = [0] * len(headline_id2word)
for (id, word) in headline_id2word.items():
    headline_len_words[id] = len(word.split(' '))
# setting word_len to 10 because 
# and args contain empty spaces at the beginning and end of the strings
headline_indices_to_remove_based_on_len = [id 
                                           for id, word_len 
                                           in enumerate(headline_len_words) 
                                           if (word_len > word_threshold )]

print(len(headline_indices_to_remove_based_on_len)) # 0

0


In [36]:
# keeping every token / word, reset freq_threshold to remove lower frequency words
freq_threshold = 0

headline_sorted_ids = [k for k, v in sorted(headline_id2freq.items(), key=lambda item: item[1])]
headline_sorted_ids.reverse()
headline_indices_to_remove_based_on_freq = [k for k, v in headline_id2freq.items() if v <= freq_threshold ]
headline_to_be_removed = list(set(headline_indices_to_remove_based_on_freq + headline_indices_to_remove_based_on_len))

In [38]:
# encode the text_lists
headline_text_rep_list = model.encode(headline_text, batch_size = 8, show_progress_bar = True)

Batches:   0%|          | 0/7017 [00:00<?, ?it/s]

In [39]:
emb_model = "phrase-bert"
print(f"Building sentence model by using {emb_model} as embedding model")

headline_uid_input_vector_list = [(i, headline_text_rep_list[i]) for i in range(len(headline_text_rep_list))]
print(f"Computed {len(headline_uid_input_vector_list)} positive examples")

Building sentence model by using phrase-bert as embedding model
Computed 56132 positive examples


In [40]:
# setting the argument num_negative_samples for negative sampling
num_neg_samples = 10 # default in the original model

headline_uid_input_vector_list_neg = []
indices = list(range(len(headline_uid_input_vector_list)))
for idx in range(len(headline_uid_input_vector_list)):
    indices_candidate = indices
    neg_indices = random.sample(indices_candidate, num_neg_samples)
    neg_samples = [headline_uid_input_vector_list[neg_i][1] for neg_i in neg_indices]
    neg_vector = np.mean(neg_samples, axis=0)
    headline_uid_input_vector_list_neg.append(neg_vector)
print(f"Computed {len(headline_uid_input_vector_list_neg)} negative examples")

Computed 56132 negative examples


In [41]:
# set up hyperparameters
headline_net_params = {}
headline_net_params["mode"] = "bert"
headline_net_params["embedding"] = headline_embs_matrix_np
headline_net_params["d_hid"] = 100
headline_net_params["num_rows"] = 50  # number of topics
headline_net_params["num_sub_topics"] = 0
headline_net_params["word_dropout_prob"] = 0.2
headline_net_params["vrev"] = headline_id2word  # idx to word map
headline_net_params["device"] = 'cuda'
headline_net_params["pred_world"] = False

In [42]:
headline_net = DictionaryAutoencoder(net_params=headline_net_params)
headline_net.to('cuda')

DictionaryAutoencoder(
  (embeddings): Embedding(76078, 768)
  (W_proj): Linear(in_features=768, out_features=100, bias=True)
  (act): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (W_att): Linear(in_features=100, out_features=768, bias=True)
  (W_out): Linear(in_features=768, out_features=76078, bias=True)
)

In [43]:
# training specs (default from original code)
num_epochs = 500
batch_size = 1024
ortho_weight = 1e-5
world_clas_weight = 0.0
headline_optim = torch.optim.Adam(headline_net.parameters(), lr=1e-4)
interpret_interval = int(np.ceil(num_epochs / 10))
h_model = 2

In [44]:
# iterating through batches
headline_batch_intervals = [
    (start, start + batch_size)
    for start in range(0, len(headline_uid_input_vector_list), batch_size)]
    # batch_intervals = batch_intervals[:100]
headline_split = int(np.ceil(len(headline_batch_intervals) * 0.9))
headline_batch_intervals_train = headline_batch_intervals[:headline_split]
headline_batch_intervals_valid = headline_batch_intervals[headline_split:]

In [45]:
import argparse
parser = argparse.ArgumentParser()
"""parser.add_argument("--lr", type=float, default=1e-4)
"""
args = parser.parse_args(args=[])
args.device = 'cuda:' + '0'
args.triplet_loss_margin = 1.0
args.triplet_loss_weight = 1.0
args.ortho_weight = 1e-5
args.neighbour_loss_weight = 1e-7
args.offset_loss_weight = 1e-4

In [46]:
# headline training
print("\n" + "=" * 70)
for epoch in range(num_epochs):
    # training
    headline_net.train()
    train_mode = True
    print(f"Epoch {epoch}")
    run_epoch(headline_net, headline_optim, headline_batch_intervals_train,
              headline_uid_input_vector_list, headline_uid_input_vector_list_neg,
              args, train_mode, h_model, epoch, 50)

    # validation
    headline_net.eval()
    train_mode = False
    with torch.no_grad():
        run_epoch(
                headline_net,
                headline_optim,
                headline_batch_intervals_valid,
                headline_uid_input_vector_list,
                headline_uid_input_vector_list_neg,
                args, 
                train_mode,
                h_model,
                epoch,
                200
        )

    if (epoch + 1) % interpret_interval == 0:
        print("Topics with probability argmax")
        topics_print_list = headline_net.rank_vocab_for_topics(
            word_embedding_matrix=headline_embs_matrix_np,
            to_be_removed=headline_to_be_removed)
        print("=" * 70)

    print()
    print()
    print()
    print("=" * 70)


Epoch 0
[TRAIN] loss: 0.9296, 0.9296, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.64 s
[VALID] loss: 0.8729, 0.8729, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.04 s



Epoch 1
[TRAIN] loss: 0.8353, 0.8353, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.56 s
[VALID] loss: 0.8055, 0.8055, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.04 s



Epoch 2
[TRAIN] loss: 0.7836, 0.7836, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.57 s
[VALID] loss: 0.7639, 0.7639, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.04 s



Epoch 3
[TRAIN] loss: 0.7469, 0.7469, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.57 s
[VALID] loss: 0.7309, 0.7309, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.04 s



Epoch 4
[TRAIN] loss: 0.7164, 0.7164, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.59 s
[VALID] loss: 0.7024, 0.7024, 0.0000, 0.0000, 0.0000 (all, tri, or, off, nei), time: 0.04 s



Epoch 5
[TRAIN] loss: 0.

In [47]:
# headline print topic list
print("Finally after training")
headline_net.eval()

print("Topics with probability argmax")
prob_over_vocab_np, topics_print_list = headline_net.rank_vocab_for_topics(
            word_embedding_matrix=headline_embs_matrix_np, to_be_removed=headline_to_be_removed
    )
print("=" * 70)

Finally after training
Topics with probability argmax
50
50
[53018 63152 39926 53191 44942 26006 65746 48651  3397 71003]
topic 0 : It lamented “calls for public recognition of the right to choose one’s gender, and of a plurality of new types of unions, in direct contradiction of the model of marriage as being between one man and one woman, which is portrayed as a vestige of patriarchal societies.”, when a new crop of younger female candidates publicly embraced their role as mothers,, recognizing a constitutional right to same-sex marriage, applied only to male couples and not to lesbians, Title IX of the Education Amendments of 1972 allows for claims only on the basis of sex, rather than gender identity, and that its policy did not violate the equal protection clause., people who are quietly not conforming to conventional notions of sexuality have been so marginalized that the issue has not been considered a human rights problem,, own mother argued that she and her same-sex partner we

In [48]:
pd.DataFrame(topics_print_list).to_csv(f"ones_TopicsToWords_NYT_2019_50_new.csv",index=False)
np.save(f"ones_argsToTopics_NYT_2019_50_new.npy",prob_over_vocab_np)

In [49]:
# after training we evaluate all the topics percentage in the dataset and rank the topics by percentage
uid_list, vector_list = zip(*headline_uid_input_vector_list)
topic_pred_list = text_to_topic(vector_list, headline_net, 'cuda')

topic_id_ranked, topic_percentage_ranked = rank_topics_by_percentage( topic_pred_list )

for rank, (topic_id, topic_percentage) in enumerate( zip(topic_id_ranked, topic_percentage_ranked)):
    print(
            f"Rank: {rank}, Topic_id: {topic_id}, Topic Words: {topics_print_list[topic_id]}, \
            Topic Percentage: {topic_percentage}"
        )

100%|██████████| 141/141 [00:00<00:00, 673.86it/s]

Rank: 0, Topic_id: 24, Topic Words: topic 24 : "it's a whole big fat hoax,”", 'think'), “I want to be careful in answering that, To see what I mean,, “literally” and “I’m serious.”, you don’t do something else?”, I’d also be asking, “What do you mean you want to change, “That is not good enough, and it is time to do things differently.”, there is something it doesn’t want you to know:, Like someone is always trying to kill you.” “I don’t want that to happen to me.”, "it's a whole big fat hoax,”", "it's a disgrace that you even asked that question,"),             Topic Percentage: 9.92
Rank: 1, Topic_id: 38, Topic Words: topic 38 : to target and take hostages but were denied entry into the main hall by hotel security guards,, One of the attackers was shot and killed by police,, trying to arrest two people suspected of stealing a bag from a man in a popular tourist district., shot and killed in front of his sister by a hooded gunman on Friday night, militants armed with knives killed the

In [50]:
prob_over_vocab_df = pd.DataFrame(prob_over_vocab_np)
prob_over_vocab_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,76068,76069,76070,76071,76072,76073,76074,76075,76076,76077
0,0.000012,0.000008,0.000016,0.000017,0.000018,0.000011,0.000016,0.000013,0.000011,0.000012,...,0.000014,0.000011,0.000012,0.000010,0.000016,0.000014,0.000008,0.000014,0.000009,0.000010
1,0.000013,0.000012,0.000009,0.000012,0.000018,0.000016,0.000010,0.000010,0.000012,0.000015,...,0.000014,0.000008,0.000013,0.000009,0.000015,0.000012,0.000018,0.000009,0.000017,0.000014
2,0.000010,0.000014,0.000010,0.000018,0.000011,0.000013,0.000011,0.000012,0.000010,0.000013,...,0.000010,0.000012,0.000009,0.000012,0.000015,0.000012,0.000016,0.000013,0.000017,0.000020
3,0.000008,0.000013,0.000016,0.000015,0.000010,0.000014,0.000010,0.000012,0.000010,0.000014,...,0.000017,0.000015,0.000011,0.000009,0.000013,0.000012,0.000016,0.000015,0.000013,0.000019
4,0.000012,0.000009,0.000016,0.000011,0.000010,0.000010,0.000022,0.000018,0.000015,0.000010,...,0.000011,0.000010,0.000010,0.000014,0.000011,0.000013,0.000009,0.000013,0.000016,0.000010


In [51]:
prob_over_vocab_df.idxmax() # for each column, find the row number of the max

0        24
1        43
2         8
3        21
4        24
         ..
76073    41
76074    30
76075    32
76076     8
76077     2
Length: 76078, dtype: int64

In [52]:
df = pd.DataFrame(prob_over_vocab_df.idxmax())
df = df.reset_index()
df

,index,0
0,0,24
1,1,43
2,2,8
3,3,21
4,4,24
...,...,...
76073,76073,41
76074,76074,30
76075,76075,32
76076,76076,8


In [53]:
# construct a dictionary with arg ids as keys and topic ids as values
argid2topic_dict = df[0].to_dict()

In [54]:
# construct a dictionary with topic ids as keys and arg ids as values
# where only one topic is assigned to each argument
topic2argid_dict = df.groupby(0)['index'].apply(list).to_dict()

In [55]:
argid2topic_dict

{0: 24,
 1: 43,
 2: 8,
 3: 21,
 4: 24,
 5: 20,
 6: 4,
 7: 33,
 8: 17,
 9: 24,
 10: 15,
 11: 19,
 12: 49,
 13: 40,
 14: 38,
 15: 31,
 16: 4,
 17: 34,
 18: 42,
 19: 2,
 20: 6,
 21: 37,
 22: 40,
 23: 30,
 24: 47,
 25: 27,
 26: 2,
 27: 12,
 28: 24,
 29: 24,
 30: 5,
 31: 21,
 32: 24,
 33: 3,
 34: 21,
 35: 0,
 36: 5,
 37: 40,
 38: 15,
 39: 0,
 40: 34,
 41: 32,
 42: 24,
 43: 24,
 44: 7,
 45: 20,
 46: 43,
 47: 19,
 48: 38,
 49: 0,
 50: 26,
 51: 21,
 52: 14,
 53: 6,
 54: 0,
 55: 0,
 56: 0,
 57: 10,
 58: 21,
 59: 36,
 60: 15,
 61: 41,
 62: 19,
 63: 20,
 64: 3,
 65: 47,
 66: 8,
 67: 32,
 68: 24,
 69: 27,
 70: 9,
 71: 23,
 72: 48,
 73: 43,
 74: 0,
 75: 46,
 76: 47,
 77: 15,
 78: 36,
 79: 16,
 80: 2,
 81: 40,
 82: 4,
 83: 24,
 84: 5,
 85: 23,
 86: 17,
 87: 6,
 88: 46,
 89: 6,
 90: 8,
 91: 6,
 92: 14,
 93: 5,
 94: 16,
 95: 6,
 96: 2,
 97: 47,
 98: 17,
 99: 32,
 100: 2,
 101: 5,
 102: 11,
 103: 29,
 104: 1,
 105: 16,
 106: 0,
 107: 21,
 108: 12,
 109: 5,
 110: 5,
 111: 38,
 112: 28,
 113: 39,
 114: 4

In [56]:
topic2argid_dict[0]

[35,
 39,
 49,
 54,
 55,
 56,
 74,
 106,
 117,
 118,
 151,
 164,
 184,
 200,
 210,
 215,
 231,
 232,
 236,
 271,
 398,
 408,
 417,
 449,
 458,
 486,
 490,
 500,
 509,
 515,
 619,
 620,
 641,
 646,
 659,
 695,
 743,
 748,
 820,
 879,
 887,
 891,
 896,
 898,
 958,
 962,
 969,
 1016,
 1028,
 1029,
 1082,
 1088,
 1108,
 1164,
 1201,
 1206,
 1259,
 1293,
 1298,
 1328,
 1436,
 1465,
 1499,
 1532,
 1558,
 1622,
 1663,
 1668,
 1706,
 1716,
 1735,
 1764,
 1766,
 1797,
 1811,
 1833,
 1848,
 1852,
 1860,
 1908,
 1929,
 1936,
 1962,
 1969,
 1981,
 2009,
 2018,
 2030,
 2051,
 2069,
 2082,
 2103,
 2110,
 2116,
 2122,
 2146,
 2205,
 2242,
 2259,
 2308,
 2316,
 2366,
 2367,
 2383,
 2391,
 2396,
 2398,
 2424,
 2432,
 2472,
 2500,
 2568,
 2589,
 2615,
 2637,
 2638,
 2647,
 2650,
 2692,
 2705,
 2713,
 2805,
 2847,
 2861,
 2866,
 2884,
 2890,
 2920,
 2933,
 2944,
 2946,
 2966,
 2974,
 2978,
 3010,
 3081,
 3092,
 3134,
 3142,
 3171,
 3176,
 3185,
 3257,
 3262,
 3316,
 3365,
 3397,
 3420,
 3421,
 3436,
 344

Construct a new dataframe with columns: sentence, arg0, arg1, arg0id, arg1id, arg0topicid, arg1topicid

In [57]:
headline_topic_df = headline.copy()
headline_topic_df.head(5)

,id,index,outlet,political_leaning,date_publish,text,args,year,arg0,arg1
0,4075871,polusa_4075871_1_1,The New York Times,LEFT,2019-01-01 00:00:00,Serious economists warned that this growth was...,('driven by the shift from fiscal austerity to...,2019,driven by the shift from fiscal austerity to w...,that this growth was a temporary lift — a “sug...
1,4075871,polusa_4075871_6_0,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('In', 'business spending.')",2019,In,business spending.
2,4075871,polusa_4075871_6_1,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('the uncertainty may be deterring', 'business...",2019,the uncertainty may be deterring,business spending.
3,4083565,polusa_4083565_5_0,The New York Times,LEFT,2019-01-01 00:00:03,George Wallace spent most of life trying to ho...,('trying to hold back the tide of change that ...,2019,trying to hold back the tide of change that Ro...,George Wallace spent most of life
4,4083565,polusa_4083565_9_0,The New York Times,LEFT,2019-01-01 00:00:03,"To defenders of the old order, America had don...","('To defenders of the old order, America had d...",2019,"To defenders of the old order, America had don...",and there was no need to open the floodgates.


In [59]:
# create arg0_id and arg1_id columns
headline_arg0id_list = []
for arg0 in headline_topic_df['arg0']:
    headline_arg0id_list.append(headline_word2id[arg0])
headline_topic_df['arg0_id'] = headline_arg0id_list

headline_arg1id_list = []
for arg1 in headline_topic_df['arg1']:
    headline_arg1id_list.append(headline_word2id[arg1])
headline_topic_df['arg1_id'] = headline_arg1id_list

In [61]:
# create columns for arg0_topicid and arg1_topicid
headline_arg0topic_list = []
for arg0 in headline_arg0id_list:
    headline_arg0topic_list.append(argid2topic_dict[arg0])
headline_topic_df['arg0_topicid'] = headline_arg0topic_list

headline_arg1topic_list = []
for arg1 in headline_arg1id_list:
    headline_arg1topic_list.append(argid2topic_dict[arg1])
headline_topic_df['arg1_topicid'] = headline_arg1topic_list

In [62]:
headline_topic_df

,id,index,outlet,political_leaning,date_publish,text,args,year,arg0,arg1,arg0_id,arg1_id,arg0_topicid,arg1_topicid
0,4075871,polusa_4075871_1_1,The New York Times,LEFT,2019-01-01 00:00:00,Serious economists warned that this growth was...,('driven by the shift from fiscal austerity to...,2019,driven by the shift from fiscal austerity to w...,that this growth was a temporary lift — a “sug...,61250,60201,36,36
1,4075871,polusa_4075871_6_0,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('In', 'business spending.')",2019,In,business spending.,8249,48109,24,36
2,4075871,polusa_4075871_6_1,The New York Times,LEFT,2019-01-01 00:00:00,"In particular, the uncertainty may be deterrin...","('the uncertainty may be deterring', 'business...",2019,the uncertainty may be deterring,business spending.,62626,48109,11,36
3,4083565,polusa_4083565_5_0,The New York Times,LEFT,2019-01-01 00:00:03,George Wallace spent most of life trying to ho...,('trying to hold back the tide of change that ...,2019,trying to hold back the tide of change that Ro...,George Wallace spent most of life,27612,63511,2,18
4,4083565,polusa_4083565_9_0,The New York Times,LEFT,2019-01-01 00:00:03,"To defenders of the old order, America had don...","('To defenders of the old order, America had d...",2019,"To defenders of the old order, America had don...",and there was no need to open the floodgates.,69893,41134,35,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56127,4079304,polusa_4079304_6_8,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('postal', 'the authorities shot and killed hi...",2019,postal,the authorities shot and killed him outside a ...,1060,47896,4,38
56128,4079304,polusa_4079304_6_9,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('indiscriminately fired from a rifle at', 'and')",2019,indiscriminately fired from a rifle at,and,33436,13146,9,24
56129,4079304,polusa_4079304_6_11,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('indiscriminately fired from a rifle at', 'th...",2019,indiscriminately fired from a rifle at,the authorities shot and killed him outside a ...,33436,47896,9,38
56130,4086266,polusa_4086266_0_0,The New York Times,LEFT,2019-08-31 17:47:03,Two people were killed and up to 20 others wer...,"('shootings in Midland, Tex., on Saturday,', '...",2019,"shootings in Midland, Tex., on Saturday,",Two people were killed and up to 20 others wer...,42291,50020,9,38


In [67]:
# save the dataframe as a csv file
topic_model_data_path = "/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/"
np.save(os.path.join(topic_model_data_path, 'ones_NYT_2019_50_new_topic_df'), headline_topic_df)
headline_topic_df.to_csv('/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/ones_NYT_2019_50_new_topic_df.csv')

In [68]:
with open( os.path.join(topic_model_data_path, 'ones_NYT_2019_50_new_topic_model.pt'), "wb") as f:
    torch.save(headline_net, f)
    print(f"Saved model at { os.path.join(topic_model_data_path) }")

Saved model at /content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/


construct a table to keep track of how many times each topic of cause has caused each topic of effect

In [69]:
headline_topic_count_df = headline_topic_df.copy()
headline_topic_count_df.tail(5)

,id,index,outlet,political_leaning,date_publish,text,args,year,arg0,arg1,arg0_id,arg1_id,arg0_topicid,arg1_topicid
56127,4079304,polusa_4079304_6_8,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('postal', 'the authorities shot and killed hi...",2019,postal,the authorities shot and killed him outside a ...,1060,47896,4,38
56128,4079304,polusa_4079304_6_9,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('indiscriminately fired from a rifle at', 'and')",2019,indiscriminately fired from a rifle at,and,33436,13146,9,24
56129,4079304,polusa_4079304_6_11,The New York Times,LEFT,2019-08-31 00:00:00,The gunman then hijacked a United States posta...,"('indiscriminately fired from a rifle at', 'th...",2019,indiscriminately fired from a rifle at,the authorities shot and killed him outside a ...,33436,47896,9,38
56130,4086266,polusa_4086266_0_0,The New York Times,LEFT,2019-08-31 17:47:03,Two people were killed and up to 20 others wer...,"('shootings in Midland, Tex., on Saturday,', '...",2019,"shootings in Midland, Tex., on Saturday,",Two people were killed and up to 20 others wer...,42291,50020,9,38
56131,4086266,polusa_4086266_1_0,The New York Times,LEFT,2019-08-31 17:47:03,He said the shootings began shortly after 4 p....,('two or possibly three gunmen targeting motor...,2019,two or possibly three gunmen targeting motoris...,the shootings began shortly after 4 p.m. local...,32224,4107,13,9


In [70]:
headline_topic_count_df = headline_topic_count_df[['arg0_topicid', 'arg1_topicid']]
headline_topic_count_df

,arg0_topicid,arg1_topicid
0,36,36
1,24,36
2,11,36
3,2,18
4,35,42
...,...,...
56127,4,38
56128,9,24
56129,9,38
56130,9,38


In [71]:
headline_topic_count_df = headline_topic_count_df.explode('arg0_topicid')
headline_topic_count_df = headline_topic_count_df.explode('arg1_topicid')
headline_topic_count_df['index'] = headline_topic_count_df.index
headline_topic_count_df

,arg0_topicid,arg1_topicid,index
0,36,36,0
1,24,36,1
2,11,36,2
3,2,18,3
4,35,42,4
...,...,...,...
56127,4,38,56127
56128,9,24,56128
56129,9,38,56129
56130,9,38,56130


In [72]:
headline_topic_count_df = headline_topic_count_df.groupby(['arg0_topicid', 'arg1_topicid'])['index'].count().unstack(fill_value=0)
headline_topic_count_df

arg1_topicid,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
arg0_topicid,,,,,,,,,,,,,,,,,,,,,
0,351,27,34,25,59,64,62,17,29,49,...,36,36,41,82,0,18,39,48,25,17
1,22,31,22,12,7,11,14,4,10,15,...,10,6,11,26,0,16,12,15,6,6
2,25,25,71,16,17,29,21,5,13,25,...,20,14,14,6,1,10,27,63,16,6
3,23,7,9,34,7,12,15,1,21,4,...,5,4,11,3,0,8,16,16,4,5
4,67,11,19,11,114,59,39,13,22,46,...,11,27,42,44,6,11,10,26,17,5
5,52,23,37,9,60,230,24,23,32,26,...,24,12,21,20,3,12,17,15,26,16
6,64,15,17,14,37,32,113,11,26,27,...,14,15,34,39,0,6,34,54,14,25
7,19,4,3,2,12,15,11,14,9,7,...,2,4,17,27,0,1,5,4,4,5
8,30,14,21,39,36,26,42,11,262,24,...,42,33,27,17,0,31,17,47,15,7


In [73]:
headline_topic_count_df.sum().sum()

56132

In [74]:
np.save(os.path.join(topic_model_data_path, 'ones_NYT_2019_50_new_causal'), headline_topic_count_df)
headline_topic_count_df.to_csv('/content/drive/MyDrive/Capstone - Causal Narratives Extraction/Phrase BERT code/ones_NYT_2019_50_new_causal.csv')